In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, resnet101
import time

In [18]:
# --- 1. Définition du modèle SE-ResNet-50 (adapté pour 10 classes) ---
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // r),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch_size, num_channels, _, _ = x.size()
        y = self.squeeze(x).view(batch_size, num_channels)
        y = self.excitation(y).view(batch_size, num_channels, 1, 1)
        return x * y.expand_as(x)

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, r=16):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.se_block = SEBlock(out_channels * self.expansion, r)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.se_block(out)

        if self.downsample is not None:
            residual = self.downsample(residual)
        
        out += residual
        out = self.relu(out)
        return out

class SEResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(SEResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False) # Changed for CIFAR-10
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )
        
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x


In [20]:
def se_resnet50(num_classes=10):
    return SEResNet(Bottleneck, [3, 4, 6, 3], num_classes)

In [21]:
# --- 2. Préparation des données CIFAR-10 ---
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [22]:

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
print(device)

cuda:0


In [24]:


# --- 3. Fonction d'entraînement et d'évaluation ---
def train_and_evaluate(model,model_name, num_epochs=100):
    # model_name = type(model).__name__
    # if model_name == "ResNet":
    #     model_name += "-50" if sum(model.layers) == 16 else "-101"

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)

    print(f"\n--- Début de l'entraînement pour {model_name} ---")
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        scheduler.step()

        # Évaluation sur le jeu de test
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

    end_time = time.time()
    training_time = end_time - start_time
    print(f"--- Entraînement terminé pour {model_name} en {training_time:.2f} secondes. ---")
    return accuracy, training_time

In [25]:


# --- 4. Exécution de la comparaison ---
results = {}

In [9]:
# Modèle 2: ResNet-50
# num_classes=10 est important pour adapter le dernier layer
resnet_50_model = resnet50(weights=None, num_classes=10) 
resnet_50_accuracy, resnet_50_time = train_and_evaluate(resnet_50_model,'ResNet-50')
results['ResNet-50'] = {'Accuracy': resnet_50_accuracy, 'Time': resnet_50_time}

# Modèle 3: ResNet-101
resnet_101_model = resnet101(weights=None, num_classes=10)
resnet_101_accuracy, resnet_101_time = train_and_evaluate(resnet_101_model,'ResNet-101')
results['ResNet-101'] = {'Accuracy': resnet_101_accuracy, 'Time': resnet_101_time}


--- Début de l'entraînement pour ResNet-50 ---
Epoch 1/100, Loss: 4.0735, Accuracy: 23.57%
Epoch 2/100, Loss: 1.9294, Accuracy: 34.25%
Epoch 3/100, Loss: 1.6914, Accuracy: 40.24%
Epoch 4/100, Loss: 1.5682, Accuracy: 44.63%
Epoch 5/100, Loss: 1.4743, Accuracy: 45.87%
Epoch 6/100, Loss: 1.3775, Accuracy: 50.53%
Epoch 7/100, Loss: 1.2801, Accuracy: 55.54%
Epoch 8/100, Loss: 1.1879, Accuracy: 56.34%
Epoch 9/100, Loss: 1.1079, Accuracy: 57.27%
Epoch 10/100, Loss: 1.0293, Accuracy: 61.16%
Epoch 11/100, Loss: 0.9862, Accuracy: 61.09%
Epoch 12/100, Loss: 0.9491, Accuracy: 63.93%
Epoch 13/100, Loss: 0.8977, Accuracy: 65.72%
Epoch 14/100, Loss: 0.8221, Accuracy: 67.91%
Epoch 15/100, Loss: 0.7772, Accuracy: 65.21%
Epoch 16/100, Loss: 0.7474, Accuracy: 66.42%
Epoch 17/100, Loss: 0.7249, Accuracy: 68.27%
Epoch 18/100, Loss: 0.6985, Accuracy: 70.29%
Epoch 19/100, Loss: 0.6774, Accuracy: 70.44%
Epoch 20/100, Loss: 0.6685, Accuracy: 67.60%
Epoch 21/100, Loss: 0.6420, Accuracy: 70.45%
Epoch 22/100, Lo

In [28]:
# Modèle 1: SE-ResNet-50
se_resnet_50_model = se_resnet50()
se_accuracy, se_time = train_and_evaluate(se_resnet_50_model,'SE-ResNet-50')
results['SE-ResNet-50'] = {'Accuracy': se_accuracy, 'Time': se_time}

# # Modèle 2: ResNet-50
# # num_classes=10 est important pour adapter le dernier layer
# resnet_50_model = resnet50(weights=None, num_classes=10) 
# resnet_50_accuracy, resnet_50_time = train_and_evaluate(resnet_50_model)
# results['ResNet-50'] = {'Accuracy': resnet_50_accuracy, 'Time': resnet_50_time}

# # Modèle 3: ResNet-101
# resnet_101_model = resnet101(weights=None, num_classes=10)
# resnet_101_accuracy, resnet_101_time = train_and_evaluate(resnet_101_model)
# results['ResNet-101'] = {'Accuracy': resnet_101_accuracy, 'Time': resnet_101_time}



--- Début de l'entraînement pour SE-ResNet-50 ---
Epoch 1/100, Loss: 2.1934, Accuracy: 39.10%
Epoch 2/100, Loss: 1.5768, Accuracy: 46.41%
Epoch 3/100, Loss: 1.3694, Accuracy: 54.66%
Epoch 4/100, Loss: 1.1554, Accuracy: 60.23%
Epoch 5/100, Loss: 1.0027, Accuracy: 64.29%
Epoch 6/100, Loss: 0.8946, Accuracy: 67.77%
Epoch 7/100, Loss: 0.8076, Accuracy: 67.38%
Epoch 8/100, Loss: 0.7090, Accuracy: 69.10%
Epoch 9/100, Loss: 0.6364, Accuracy: 73.90%
Epoch 10/100, Loss: 0.5730, Accuracy: 75.60%
Epoch 11/100, Loss: 0.5226, Accuracy: 74.68%
Epoch 12/100, Loss: 0.4741, Accuracy: 73.23%
Epoch 13/100, Loss: 0.4434, Accuracy: 76.38%
Epoch 14/100, Loss: 0.4187, Accuracy: 78.20%
Epoch 15/100, Loss: 0.4022, Accuracy: 79.85%
Epoch 16/100, Loss: 0.3800, Accuracy: 79.70%
Epoch 17/100, Loss: 0.3663, Accuracy: 79.15%
Epoch 18/100, Loss: 0.3533, Accuracy: 77.87%
Epoch 19/100, Loss: 0.3359, Accuracy: 78.24%
Epoch 20/100, Loss: 0.3235, Accuracy: 78.77%
Epoch 21/100, Loss: 0.3123, Accuracy: 79.19%
Epoch 22/100,

In [29]:
# --- 5. Affichage des résultats ---
print("\n" + "="*50)
print("             RÉSUMÉ DE LA COMPARAISON")
print("="*50)
for model_name, data in results.items():
    print(f"Modèle: {model_name}")
    print(f"  Précision finale: {data['Accuracy']:.2f}%")
    print(f"  Temps d'entraînement: {data['Time']:.2f} secondes")
    print("-" * 30)


             RÉSUMÉ DE LA COMPARAISON
Modèle: ResNet-50
  Précision finale: 80.05%
  Temps d'entraînement: 3962.36 secondes
------------------------------
Modèle: ResNet-101
  Précision finale: 78.17%
  Temps d'entraînement: 6141.50 secondes
------------------------------
Modèle: SE-ResNet-50
  Précision finale: 88.52%
  Temps d'entraînement: 10947.67 secondes
------------------------------
